<a href="https://colab.research.google.com/github/alejovillores/tp2_spark_villores_alejo/blob/main/ejercicio_8_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ejercicio 8
Considerando el pagelink_sample.csv, usando una representación de grafos obtener
aquellos contenidos que tienen “relaciones no correspondidas”. Entendemos como
funciona una relación correspondida con un ejemplo: Si el contenido A tiene un link
al B, pero B no tiene un link a A, podemos decir que B tiene una relación no
correspondida con A.

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.3 MB 37 kB/s 
     |████████████████████████████████| 198 kB 60.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=ec1436334ef24c06cb76fc95796fb76950fe2cf42ec27317935f808894dd3b84
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
!pip install pyarrow

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
pagelinks = pd.read_parquet('/content/drive/MyDrive/2C2021/Datos/Trabajo Practico 2/pagelinks_sample.csv')
pagelinks.head()

,pl_from,pl_namespace,pl_title,pl_from_namespace
0,4840492,10,Bandera_cÃ³digo_del_COI_TPE,2
1,950948,0,Esteban_DuÅ¡an,0
2,6418187,0,Insecta,0
3,3436623,0,Pleasure_to_Kill,0
4,4713066,0,DamiÃ¡n_Oliver,0


In [ ]:
contents = pd.read_parquet('/content/drive/MyDrive/2C2021/Datos/Trabajo Practico 2/contents.csv')
contents

,title,id,namespace,revision_id,parent_revision,revision_timestamp,revisor_username,revisor_id,revisor_ip,revisor_comment
0,Wikipedia:Artículos solicitados,5,4,132533307,132516631.0,2021-01-19T18:40:34Z,Metrónomo,1708233.0,None,Así esta mejor
1,Andorra,7,0,138035057,138027475.0,2021-08-31T18:27:01Z,None,NaN,2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2,Añadir contenido
2,Argentina,10,0,137966826,137825165.0,2021-08-28T17:17:54Z,Traitrich,4642283.0,None,Reemplazos con [[Usuario:Benjavalero/Replacer|...
3,Geografía de Andorra,15,0,137986788,133666383.0,2021-08-29T17:16:00Z,Lojwe,5338508.0,None,Reemplazos con [[Usuario:Benjavalero/Replacer|...
4,Demografía de Andorra,17,0,130869554,129413188.0,2020-11-12T19:56:15Z,Jialxv,3615366.0,None,Correcciones ortográficas con [[Usuario:Benjav...
...,...,...,...,...,...,...,...,...,...,...
4132159,Cruz nestoriana,10010593,0,138052016,138051910.0,2021-09-01T12:06:00Z,Copydays,474907.0,None,termino traducción y quito plantilla en desarr...
4132160,Manifestación del orgullo gay de 1977 en Barce...,10010595,0,138051859,NaN,2021-09-01T11:52:28Z,Crls9011,4032712.0,None,Crls9011 trasladó la página [[Manifestación de...
4132161,Rocasaurus,10010596,0,138051902,NaN,2021-09-01T11:57:06Z,Gabriel Solo,158577.0,None,Gabriel Solo trasladó la página [[Rocasaurus]]...
4132162,Medalla Conmemorativa del 50.º Aniversario de ...,10010598,0,138051961,NaN,2021-09-01T12:01:38Z,Términus,5429951.0,None,Términus trasladó la página [[Medalla Conmemor...


In [ ]:
pagelinks_reduce = pagelinks.loc[pagelinks['pl_title'].isin(contents['title'])]


,pl_from,pl_namespace,pl_title,pl_from_namespace
0,4840492,10,Bandera_cÃ³digo_del_COI_TPE,2
1,950948,0,Esteban_DuÅ¡an,0
2,6418187,0,Insecta,0
3,3436623,0,Pleasure_to_Kill,0
4,4713066,0,DamiÃ¡n_Oliver,0


In [ ]:
pagelinks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000000 entries, 0 to 79999999
Data columns (total 4 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   pl_from            int64 
 1   pl_namespace       int64 
 2   pl_title           object
 3   pl_from_namespace  int64 
dtypes: int64(3), object(1)
memory usage: 2.4+ GB


In [ ]:
pagelinks_reduce.head()

,pl_from,pl_namespace,pl_title,pl_from_namespace
2,6418187,0,Insecta,0
8,6571966,0,IMDb,0
12,5461523,4,Vandalismo,3
13,1098340,0,Wikidata,0
16,6224910,10,Referencias,3


In [ ]:
pagelinks_reduce.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20572932 entries, 2 to 79999997
Data columns (total 4 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   pl_from            int64 
 1   pl_namespace       int64 
 2   pl_title           object
 3   pl_from_namespace  int64 
dtypes: int64(3), object(1)
memory usage: 784.8+ MB


In [ ]:
pagelinks_reduce.to_parquet('/content/drive/MyDrive/2C2021/Datos/Trabajo Practico 2/pagelinks_sample.parquet')

In [ ]:
sqlContext = SQLContext(sc)
df_contenidos = sqlContext.read.parquet('/content/drive/MyDrive/2C2021/Datos/Trabajo Practico 2/contents.csv', header=True, inferSchema=True)
rdd_contenidos = df_contenidos.rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
sqlContext = SQLContext(sc)
df_pagelink = sqlContext.read.parquet('/content/drive/MyDrive/2C2021/Datos/Trabajo Practico 2/pagelinks_sample.parquet', header=True, inferSchema=True)
rdd_pagelinks = df_pagelink.rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Combierto mis valores de page links a clave valor, donde la clave es el titulo que comparte con contents

In [ ]:
rdd_pagelink_title = rdd_pagelinks.filter(lambda x: x.pl_from != None).map(lambda x: (x.pl_title,x.pl_from))

In [ ]:
rdd_pagelink_title.take(3)

[('Insecta', 6418187), ('IMDb', 6571966), ('Vandalismo', 5461523)]

In [ ]:
rdd_contenidos_title_id = rdd_contenidos.filter(lambda x: x.id != None).map(lambda x: (x.title,x.id))

In [ ]:
rdd_contenidos_title_id.take(3)

[('Wikipedia:Artículos solicitados', 5), ('Andorra', 7), ('Argentina', 10)]

In [ ]:
join_data = rdd_pagelink_title.join(rdd_contenidos_title_id)

In [ ]:
join_data.take(3)

[('Trochamminoidea', (7313489, 5782285)),
 ('Trochamminoidea', (6364333, 5782285)),
 ('Trochamminoidea', (7313440, 5782285))]

A traves de los nodos del grafo busco lo pedido


In [ ]:
nodos = join_data.map(lambda x: (x[1][0],x[1][1]))

In [ ]:
nodos.take(3)

[(596996, 4050588), (9474473, 4050588), (7061488, 4050588)]

In [ ]:
nodos_original_ordenado = nodos.map(lambda x: (tuple(sorted(x)),x))

In [ ]:
nodos_original_ordenado.take(3)

[((596996, 4050588), (596996, 4050588)),
 ((4050588, 9474473), (9474473, 4050588)),
 ((4050588, 7061488), (7061488, 4050588))]

In [ ]:
rdd_relaciones = nodos.map(lambda x: tuple(sorted(x))).map(lambda x:(x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1] == 1)

In [ ]:
nodos_original_ordenado.join(rdd_relaciones).take(5)

[((10256, 1957010), ((1957010, 10256), 1)),
 ((2587481, 2657647), ((2587481, 2657647), 1)),
 ((5206, 7089920), ((7089920, 5206), 1)),
 ((312036, 312866), ((312036, 312866), 1)),
 ((41397, 7146771), ((7146771, 41397), 1))]